In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        subclasses = self.get_subclasses()
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self):
        subclasses = [(self.extract_ID(a), self.extract_ID(b), c, d) for (a,b,c,d) in self.subclasses]
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [4]:
USE_folder = "/home/vlead/USE"
alignment_folder = "AML-conference/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2'), doc.getElementsByTagName('measure')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource'), c.firstChild.nodeValue) for (a,b,c) in ls])
    return alignments

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)

# ra_anatomy_coded = load_alignments("../Anatomy/Alignments/")
# ra_anatomy = []
# ont1 = Ontology("../Anatomy/Ontologies/mouse.owl")
# ont2 = Ontology("../Anatomy/Ontologies/human.owl")
# for elem in ra_anatomy_coded:
#     pre1, pre2 = elem[0].split("#")[0].split(".")[0].split("/")[-1], elem[1].split("#")[0].split(".")[0].split("/")[-1]
#     elem1, elem2 = elem[0].split("#")[-1], elem[1].split("#")[-1]
#     ra_anatomy.append(( pre1 + "#" + ont1.mapping_dict[elem1], pre2 + "#" + ont2.mapping_dict[elem2]))

gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]
# gt_mappings.extend(ra_anatomy)


ontologies_in_alignment = pickle.load(open("../data_generic.pkl", "rb"))[-1][:-1]
# ontologies_in_alignment += [["../Anatomy/Ontologies/human.owl", "../Anatomy/Ontologies/mouse.owl"]]


In [9]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))

    all_mappings.extend([(l[0].split(".")[0].split("/")[-1] + "#" + el[0], l[1].split(".")[0].split("/")[-1] + "#" + el[1]) for el in mappings])

aml_data = {mapping: 0 for mapping in all_mappings}
for mapping in set(gt_mappings):
    aml_data[tuple(mapping[:2])] = mapping[-1]

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [25]:
Counter(sorted([float(el[1]) for el in aml_data.items() if el[1]]))

Counter({0.6: 4,
         0.6075: 3,
         0.6123: 1,
         0.6194: 1,
         0.6248: 1,
         0.627: 1,
         0.6362: 1,
         0.6534: 1,
         0.6579: 1,
         0.66: 7,
         0.6621: 1,
         0.6656: 4,
         0.6674: 1,
         0.6857: 1,
         0.6939: 2,
         0.7128: 1,
         0.7316: 1,
         0.7351: 7,
         0.7361: 1,
         0.7367: 3,
         0.7635: 1,
         0.7679: 1,
         0.7753: 1,
         0.78: 1,
         0.7841: 1,
         0.7875: 1,
         0.8036: 4,
         0.8168: 1,
         0.8338: 1,
         0.846: 2,
         0.8571: 1,
         0.8821: 1,
         0.9603: 12,
         0.9619: 2,
         0.9653: 1,
         0.9801: 65,
         0.9802: 2,
         0.9804: 1,
         0.9807: 1,
         0.9809: 1,
         0.9812: 2,
         0.9813: 1,
         0.9823: 1,
         0.9824: 1,
         0.9827: 1,
         0.9828: 1,
         0.9835: 1,
         0.9836: 3,
         0.9847: 1,
         0.9848: 2,
       

In [8]:
# Abbrevation resolution preprocessing

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Chair', 'Program_Committee') PC Program_Committee
left ('PC_Chair', 'Program_Committee_member') PCM Program_Committee
left ('PC_Chair', 'Program_Chair') PC Program_Chair
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Orga

In [12]:

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten([(a,b,c) for (a,b,c,d) in ont.get_triples()])))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]

# Resolving abbreviations to full forms
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    final_list = []
    # Lowering case except in abbreviations
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", word):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    concept = " ".join(final_list)
    inp_resolved.append(concept)


url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

inp_spellchecked = []
for concept in inp_resolved:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        final_list = []
        for word in concept.split(" "):
            if not re.search("[A-Z][A-Z]+", concept):
                final_list.append(word.lower())
            else:
                final_list.append(word)
        resolved = " ".join(final_list)
#         print (resolved, "suggestion", response)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word.lower(), response["corrections"][word][0].lower())
                
        
        print (concept, resolved)
        inp_spellchecked.append(resolved)
    else:
        inp_spellchecked.append(concept)

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

# stopwords = ["has", "is", "a", "an", "the"]
stopwords = ["has"]
inp_stemmed = []
for elem in inp_spellchecked:
    words = " ".join([word for word in elem.split() if word not in stopwords])
    words = words.replace("-", " ")
    inp_stemmed.append(words)


embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_stemmed)))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}



list index out of range


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [70]:
emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf = pickle.load(open("../Input/data_aml_uniqpath.pkl", "rb"))[2:5]

In [141]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].split(".")[0]

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [OrderedSet([tuple([ont + "#" + node for node in path]) for path in nbr_type])
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# # ontologies_in_alignment_rev = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment]
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # neighbours_dicts

edas
list index out of range
confOf
conference
sigkdd
cmt
ekaw
iasted


In [49]:
data_conf, emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf, gt_mappings_conf, neighbours_dicts_conf = data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts

In [151]:
data_german, emb_indexer_german, emb_indexer_inv_german, emb_vals_german, gt_mappings_german, neighbours_dicts_german, ontologies_in_alignment_rev, max_types_german, max_paths_german, max_pathlen_german = pickle.load(open("Input/data_german_dataset_demarcated_bagofnbrs.pkl", "rb"))
neighbours_dicts_conf = neighbours_dicts

4

In [71]:
emb_indexer = {}
for key in emb_indexer_conf:
    emb_indexer[key] = emb_indexer_conf[key]
for key in emb_indexer_german:
    if key != "<UNK>":
        emb_indexer[key] = len(emb_indexer_conf) - 1 + emb_indexer_german[key]
emb_indexer_inv = {}
for key in emb_indexer:
    emb_indexer_inv[emb_indexer[key]] = key

In [149]:
neighbours_dicts = {}
for key in neighbours_dicts_conf:
    neighbours_dicts[key] = neighbours_dicts_conf[key]
for key in neighbours_dicts_german:
    neighbours_dicts[key] = neighbours_dicts_german[key]
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])

In [80]:
f = open("Input/data_conf_oaei_german_aml.pkl", "wb")
pickle.dump([data_conf, data_german, aml_data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, max_paths, max_pathlen, max_types, ontologies_in_alignment], f)


In [14]:
data_conf, data_german, _, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, max_paths, max_pathlen, max_types, ontologies_in_alignment = pickle.load(open("Input/data_conf_oaei_german_aml_bagofnbrs.pkl", "rb"))
pickle.dump([data_conf, data_german, aml_data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, max_paths, max_pathlen, max_types, ontologies_in_alignment], open("Input/data_conf_oaei_german_aml_bagofnbrsC.pkl", "wb"))


In [150]:
f = open("Input/data_conf_oaei_german_bagofnbrs.pkl", "wb")
# pickle.dump([data_conf, data_german, emb_indexer_german, emb_indexer_inv_german, emb_vals_german, emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf, neighbours_dicts_german, neighbours_dicts_conf, max_paths_german, max_pathlen_german, max_types_german, max_paths_conf, max_pathlen_conf, max_types_conf, ontologies_in_alignment], f)
pickle.dump([data_conf, data_german, emb_indexer_german, emb_indexer_inv_german, emb_vals_german, emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf, neighbours_dicts, max_paths, max_pathlen, max_types, ontologies_in_alignment], f)

In [111]:
data_conf, data_german, aml_data, emb_indexer, emb_indexer_inv, emb_vals, _, _, _, _, ontologies_in_alignment = pickle.load(open("Input/data_conf_oaei_german_aml.pkl", "rb"))
f = open("Input/data_conf_oaei_german_aml_bagofnbrs.pkl", "wb")
pickle.dump([data_conf, data_german, aml_data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts, max_paths, max_pathlen, max_types, ontologies_in_alignment], f)


In [146]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append(e2)
            neighbours_dict[e2][2].append(e1)
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append(e2)
            neighbours_dict[e2][3].append(e1)
        elif d == "Subclass":
            neighbours_dict[e2][1].append(e1)
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].split(".")[0]

    
    for entity in neighbours_dict:
        neighbours_dict[entity][1] = [neighbours_dict[entity][1]]
        neighbours_dict[entity][2] = [neighbours_dict[entity][2]]
        neighbours_dict[entity][3] = [neighbours_dict[entity][3]]
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [OrderedSet([tuple([ont + "#" + node for node in path]) for path in nbr_type])
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
                                for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
                                for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# # # ontologies_in_alignment_rev = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment]
# # f = open("data_aml_uniqpath.pkl", "wb")
# # pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

edas
list index out of range
confOf
conference
sigkdd
cmt
ekaw
iasted


In [147]:
neighbours_dicts_conf = neighbours_dicts

In [132]:
_, data_conf, emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf, gt_mappings_conf, _, ontologies_in_alignment_conf = pickle.load(open("../Input/data_aml.pkl", "rb"))
pickle.dump([aml_data, data_conf, emb_indexer_conf, emb_indexer_inv_conf, emb_vals_conf, gt_mappings_conf, neighbours_dicts_pathpadded, ontologies_in_alignment_conf], open("Input/data_conf_oaei_aml_bagofnbrs.pkl", "wb"))

In [154]:
# max_paths
sorted(Counter(flatten([[len(nbr_type) for nbr_type in elem] for elem in pickle.load(open("Input/data_conf_oaei_german_bagofnbrs.pkl", "rb"))[-5].values()])).items())

[(0, 93), (1, 14503), (2, 12)]

In [155]:
sorted(Counter(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in pickle.load(open("Input/data_conf_oaei_german_bagofnbrs.pkl", "rb"))[-5].values()])).items())

[(0, 9715),
 (1, 453),
 (2, 452),
 (3, 758),
 (4, 1417),
 (5, 1097),
 (6, 382),
 (7, 107),
 (8, 47),
 (9, 18),
 (10, 9),
 (11, 7),
 (12, 10),
 (13, 9),
 (14, 1),
 (15, 5),
 (16, 4),
 (17, 4),
 (18, 1),
 (19, 2),
 (20, 5),
 (21, 6),
 (22, 3),
 (23, 2),
 (24, 2),
 (25, 2),
 (26, 3),
 (29, 1),
 (38, 2),
 (42, 1),
 (48, 1),
 (49, 1)]

In [91]:
def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))

attended_path = torch.bmm(a.reshape(-1, 1, 512), b.reshape(-1, 512, 1))
attended_path = attended_path.reshape(-1, 4, 22, 6)
path_weights = masked_softmax(torch.sum(attended_path, dim=-1))
path_weights.shape
# best_path = torch.sum(path_weights[:, :, :, None, None] * a, dim=2)

torch.Size([10, 4, 22])

In [427]:
indices = d[(..., ) + (None, ) * 3].expand(-1, -1, -1, 6, 512)
e = torch.gather(b, 2, indices).squeeze(2)

f = torch.sum(a[:,None,None,:] *e,dim=-1).unsqueeze(-1)

g = (f*e)

h = torch.sum((v[None,None,:,None] * g), dim=2)

In [114]:
c = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,1,6,512))
d = b * c


In [122]:
e = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)


In [123]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load("~/", mmap='r')

(tensor([[[[-2.6975e-02,  1.1427e+00,  4.1698e-01,  ..., -8.3755e+00,
            -4.7765e-01, -2.1633e-01],
           [ 2.8020e-01, -6.6962e-01,  1.4016e+00,  ...,  7.5675e-04,
             2.2802e-01, -1.7932e+00],
           [-1.1675e-01,  2.9058e-02,  1.4350e-01,  ..., -1.8113e+00,
            -9.6346e-01,  2.6943e+00],
           [ 9.3339e-02,  5.5481e-01,  5.7394e-01,  ...,  6.9919e-01,
            -3.4617e-01,  2.7778e-01],
           [-1.4753e-01, -4.3234e-01,  1.7809e+00,  ..., -1.4585e+00,
            -7.7736e-01,  1.1985e-01],
           [-5.5448e-01,  8.2052e-01,  1.4681e-03,  ..., -3.1119e-02,
            -1.6508e-01, -6.0103e-01]],
 
          [[ 1.1321e-01,  1.9631e-01,  1.0589e+00,  ...,  4.4958e+00,
             4.9575e-02,  2.7624e-01],
           [-2.5041e-01,  1.0904e+00,  1.4865e+00,  ..., -3.6319e-01,
            -4.8349e-01,  6.8312e-01],
           [-2.0786e+00,  5.2171e-03, -1.3269e-02,  ...,  1.5515e-01,
             1.1037e+00, -2.8357e-01],
           [-1.6

In [124]:
d[0]

tensor([[[[-2.6975e-02,  1.1427e+00,  4.1698e-01,  ..., -8.3755e+00,
           -4.7765e-01, -2.1633e-01],
          [ 2.8020e-01, -6.6962e-01,  1.4016e+00,  ...,  7.5675e-04,
            2.2802e-01, -1.7932e+00],
          [-1.1675e-01,  2.9058e-02,  1.4350e-01,  ..., -1.8113e+00,
           -9.6346e-01,  2.6943e+00],
          [ 9.3339e-02,  5.5481e-01,  5.7394e-01,  ...,  6.9919e-01,
           -3.4617e-01,  2.7778e-01],
          [-1.4753e-01, -4.3234e-01,  1.7809e+00,  ..., -1.4585e+00,
           -7.7736e-01,  1.1985e-01],
          [-5.5448e-01,  8.2052e-01,  1.4681e-03,  ..., -3.1119e-02,
           -1.6508e-01, -6.0103e-01]],

         [[ 1.1321e-01,  1.9631e-01,  1.0589e+00,  ...,  4.4958e+00,
            4.9575e-02,  2.7624e-01],
          [-2.5041e-01,  1.0904e+00,  1.4865e+00,  ..., -3.6319e-01,
           -4.8349e-01,  6.8312e-01],
          [-2.0786e+00,  5.2171e-03, -1.3269e-02,  ...,  1.5515e-01,
            1.1037e+00, -2.8357e-01],
          [-1.6813e-01, -3.7349e-01

In [127]:
ctimes, dtimes = [], []
for i in range(100):
    a = torch.randn((10,4,22,6,512))
    b = torch.randn((10,4,1,6,512))
    t = time.time()
    c = a * b
    ctimes.append(time.time()-t)
    t = time.time()
    d = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)
    dtimes.append(time.time()-t)

In [130]:
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))
c = a * torch.sum(b,dim=2).unsqueeze(2)

In [134]:
np.array([(a[0][0][0] * b[0][0][i]).numpy() for i in range(22)]).shape

(22, 6, 512)

In [9]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
for i in list(range(0, len(all_ont_pairs), 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0] + "-" + ont_pair[1]
        java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s conference_ontologies/" + a + ".owl" + \
                            " -t conference_ontologies/" + b + ".owl -o AML-test-results/" + c + ".rdf -a"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

['confOf-sigkdd.rdf', 'iasted-sigkdd.rdf', 'cmt-ekaw.rdf']
0.8275862068965517 0.7272727272727273 0.7741935483870968 0.7453416149068324 0.8053691275167786
['confOf-iasted.rdf', 'conference-edas.rdf', 'cmt-sigkdd.rdf']
0.8148148148148148 0.5789473684210527 0.6769230769230768 0.6145251396648045 0.7534246575342465
['ekaw-sigkdd.rdf', 'conference-sigkdd.rdf', 'conference-confOf.rdf']
0.78125 0.6097560975609756 0.684931506849315 0.6377551020408163 0.7396449704142012
['confOf-edas.rdf', 'edas-iasted.rdf', 'cmt-conference.rdf']
0.7941176470588235 0.5094339622641509 0.6206896551724137 0.548780487804878 0.7142857142857143
['edas-sigkdd.rdf', 'conference-iasted.rdf', 'ekaw-iasted.rdf']
0.7916666666666666 0.48717948717948717 0.6031746031746031 0.5277777777777778 0.7037037037037036
['cmt-confOf.rdf', 'cmt-edas.rdf', 'edas-ekaw.rdf']
0.8181818181818182 0.5192307692307693 0.6352941176470589 0.5601659751037344 0.733695652173913
['conference-ekaw.rdf', 'confOf-ekaw.rdf', 'cmt-iasted.rdf']
0.79545454545

In [168]:
ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment][:-1] + [["human", "mouse"]]

In [420]:
nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)])).shape

torch.Size([6])

In [20]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [20]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

(167, 1240)

In [445]:
def add1(elem):
    return elem + 1

v = np.random.random((3,2,3))
v, np.vectorize(add1)([[1,2,3],[1,2,3]])

(array([[[0.86559937, 0.45132089, 0.79395411],
         [0.87339692, 0.68265425, 0.69084263]],
 
        [[0.08953082, 0.95578629, 0.06638403],
         [0.63675515, 0.08652004, 0.47466858]],
 
        [[0.38824256, 0.51993905, 0.32977087],
         [0.95501009, 0.26236804, 0.35740414]]]),
 array([[2, 3, 4],
        [2, 3, 4]]))

In [8]:
neighbours_dicts

{'iasted': {'iasted#Tutorial': [['iasted#Lecture',
    'iasted#Activity',
    'iasted#Conference_activity',
    'iasted#Lecture',
    '<UNK>',
    '<UNK>',
    '<UNK>'],
   ['<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>'],
   ['<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>'],
   ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>']],
  'iasted#Trip_city': [['iasted#City',
    'iasted#City',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>'],
   ['iasted#Conference_Hiker',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>',
    '<UNK>

In [83]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




registeered applicant registered applicant
technically organised by technically organized by
ngo no
sponzorship sponsorship


In [77]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

{'original': 'technically Organised By',
 'suggestion': 'technically Organized By',
 'corrections': {'Organised': ['Organized',
   'Organist',
   'Organism',
   'Organizes',
   'Disorganize',
   'Organize',
   'Agonized']}}

In [ ]:
inp_spellchecked[730], inp[731]

In [78]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

7796

In [14]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

[['confOf', 'sigkdd'],
 ['iasted', 'sigkdd'],
 ['cmt', 'ekaw'],
 ['confOf', 'iasted'],
 ['conference', 'edas'],
 ['cmt', 'sigkdd'],
 ['ekaw', 'sigkdd'],
 ['conference', 'confOf'],
 ['conference', 'sigkdd'],
 ['confOf', 'edas'],
 ['cmt', 'conference'],
 ['edas', 'iasted'],
 ['conference', 'iasted'],
 ['edas', 'sigkdd'],
 ['ekaw', 'iasted'],
 ['cmt', 'edas'],
 ['edas', 'ekaw'],
 ['cmt', 'confOf'],
 ['confOf', 'ekaw'],
 ['conference', 'ekaw'],
 ['cmt', 'iasted']]

In [72]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

[['confOf#Organization', 'sigkdd#Organizator', '1', '2', '3', '4'],
 ['iasted#Document', 'sigkdd#Document', '5', '6', '78', '8']]

In [34]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

left ('Chair_PC', 'Program_Chair') PC Program_Chair
left ('Chair_PC', 'Program_Committee') PC Program_Committee
left ('Chair_PC', 'Program_Committee_member') PCM Program_Committee
left ('Member_PC', 'Program_Chair') PC Program_Chair
left ('Member_PC', 'Program_Committee') PC Program_Committee
left ('Member_PC', 'Program_Committee_member') PCM Program_Committee
left ('Chair_PC', 'Presenter_city') PC Presenter_city
left ('Member_PC', 'Presenter_city') PC Presenter_city
left ('OC_Member', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Member', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Member', 'Program_Chair') PC Program_Chair
left ('PC_Member', 'Program_Committee') PC Program_Committee
left ('PC_Member', 'Program_Committee_member') PCM Program_Committee
left ('OC_Chair', 'Organizing_Committee_member') OCM Organizing_Committee
left ('OC_Chair', 'Organizing_Committee') OC Organizing_Committee
left ('PC_Chair', 'Program_Chair') PC Program_Chair
left ('PC_C

In [48]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [27]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

0.8085169792175293

In [159]:
len(pickle.load(open("../Input/data_german_dataset.pkl","rb"))[0])

130000

In [53]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [61]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

['pay',
 'rejected by',
 'Registration SIGMOD Member',
 'is connected with',
 'NGO',
 'overhead projector',
 'name of conference',
 'call for participation',
 'coffee break',
 'scientifically organised by',
 'volunteer',
 'publisher',
 'regular',
 'add program committee member',
 'contact email',
 'part of event',
 'dinner banquet',
 'social program',
 'decision',
 'is sent by',
 'was a committee chair of',
 'organisation',
 'paper',
 'assign external reviewer',
 'was a member of',
 'has cost amount',
 'session chair',
 'single level conference',
 'hotel',
 'deadline hotel reservation',
 'country',
 'important dates',
 'paper',
 'has parts',
 'event',
 'paper due on',
 'is the 1th part of',
 'has surname',
 'wireless communications topic',
 'two level conference',
 'has first name',
 'bid',
 'is designed for',
 'author',
 'double hotel room',
 'review form',
 'presentation',
 'subject area',
 'positive integer',
 'deadline abstract submission',
 'has a degree',
 'is present in',
 'rela

In [23]:
Ontology("conference_ontologies/conference.owl").triples

[('Conference_proceedings', 'string', 'has_a_name'),
 ('Committee_member', 'Committee', 'was_a_member_of'),
 ('Review', 'Conference_document', 'subclass_of'),
 ('Review_preference', 'Reviewer', 'belongs_to_reviewers'),
 ('Information_for_participants', 'Conference_document', 'subclass_of'),
 ('Regular_author', 'Conference_contributor', 'subclass_of'),
 ('Late_paid_applicant', 'Paid_applicant', 'subclass_of'),
 ('Active_conference_participant', 'Presentation', 'gives_presentations'),
 ('Invited_speaker', 'Conference_contributor', 'subclass_of'),
 ('Program_committee', 'Conference_volume', 'was_a_program_committee_of'),
 ('Topic', 'Review_preference', 'has_been_assigned_a_review_reference'),
 ('Conference_volume', 'Important_dates', 'has_important_dates'),
 ('Rejected_contribution', 'Reviewed_contribution', 'subclass_of'),
 ('Contribution_co-author', 'Regular_author', 'subclass_of'),
 ('Active_conference_participant', 'Conference_contributor', 'subclass_of'),
 ('Presentation', 'Abstract'

122893